# Iron Speciation Model
This assumes that the system is sulphide-limited, with plenty of highly-reactive iron available. Therefore, the amount of pyrite produced is equal to the amount of H<sub>2</sub>S that entered the system (divided by 2, since 1 FeS<sub>2</sub> is made from 2 H<sub>2</sub>S). I assume a constant sulphide flux, $q_\mathrm{H_2S}$, so pyrite is being generated at a constant rate $d_\mathrm{py}$ for a time $t$.

$$\frac{\partial\mathrm{[Fe_{py}]}}{\partial t} = d_\mathrm{py}$$

$$\mathrm{[Fe_{py}]}=\int d_\mathrm{py}\,\mathrm{d}t = d_\mathrm{py}\,t+\mathrm{[Fe_{py}]}_0$$

$$\left(\frac{\mathrm{[Fe_{py}]}}{\mathrm{[Fe_{HR}]}}\right)_t = \left(\frac{\mathrm{[Fe_{py}]}}{\mathrm{[Fe_{HR}]}}\right)_0 + \frac{d_\mathrm{py}\,t}{\mathrm{[Fe_{HR}]}}$$

Note that this is equivalent to an exponentially-decaying sulphide flux integrated over infinite time:
$$\frac{\partial\mathrm{[Fe_{py}]}}{\partial t} = d_0 e^{-\lambda t}$$
where $d_\mathrm{py}\cdot t = \frac{d_0}{\lambda}$.

To convert $q_\mathrm{H_2S}$ to $d_\mathrm{py}$:
$$d_\mathrm{py} = \frac{q_\mathrm{H_2S} \cdot M_r(\mathrm{Fe})}{2\rho_s}$$
where $M_r(\mathrm{Fe})$ is the relative atomic mass of iron (55.845 g/mol), and $\rho_s$ is the density of freshly-deposited sediment (about 1.7 g/cm³). This means that a typical H₂S flux of 200 pmol cm⁻³ day⁻¹ is equivalent to a pyrite production rate of 0.12 wt% per ka.

In [1]:
from plotly_default import go, graph_config, sel_trace
import pandas as pd
import numpy as np
import ipywidgets as wdg
from plotly.subplots import make_subplots
import plotly.colors as pcol

In [2]:
def wt_to_M(wt, Mr=55.845, density=1.7):
    '''Converts a weight percent of a substance with molar mass Mr (default iron) into a molar concentration
    in a sediment of specified density (default 1.7 g/cm³)'''
    # g/dm³  = 1000 * g/cm³ = 1000 * density * wt% / 100 = 10 * density * wt%
    return 10. * density * wt / Mr

def M_to_wt(M, Mr=55.845, density=1.7):
    return M * Mr / (10. * density)

slider_style=dict(description_width='initial')

In [3]:
M_to_wt(0.16)

0.5256

In [4]:
# Read in data
species_types=pd.read_csv('./Controls/Species_types.csv')
log_df=pd.read_csv('./Data/Log_data_combined.csv',index_col=0)

try:
    log_df.insert(loc=9, column='Constant', value=1) #
except ValueError:
    pass # do nothing if column already exists

data_types = species_types.append(pd.DataFrame({'header': ['Constant', 'height'], 'label': ['', '<i>h</i> / m']}), ignore_index=True, sort=False)

axis_options = list(data_types.header)
size_options = [i for i in axis_options if i != 'd34S']

def axis_mode(species):
    if species.startswith('Fe') or species.startswith('py_to') or species.startswith('T'):
        return 'tozero'
    else:
        return 'normal'

C:\Users\Peter\AppData\Local\Temp\ipykernel_12040\2094219562.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_types = species_types.append(pd.DataFrame({'header': ['Constant', 'height'], 'label': ['', '<i>h</i> / m']}), ignore_index=True, sort=False)


In [5]:
# categories = list(set(log_df.category))
# categories = ['Stratigraphic sequence', 'Pervasively pyritised regions', 'Bed 22']
cat_symbols = {'Mudstone': 'circle',
               'Carbonate': 'star',
               'Pervasively pyritised regions': 'square',
               'Bed 22': 'diamond'}
categories = list(cat_symbols.keys())

def init_biv_plot():
    plot = go.FigureWidget()
    
    for category in categories:   
        plot.add_trace(go.Scatter(mode='markers',
                                  marker_line={'color': 'Black', 'width': 1},
                                  marker_symbol=cat_symbols[category],
                                  marker_coloraxis='coloraxis',
                                  name=category))
        
    # Add diagonal line
    plot.add_shape(type='line', x0=-100, y0=-100, x1=100, y1=100, visible=False, line_width=0.5, name='Diagonal')

    plot.update_layout(height=700, width=740,
                       legend=dict(orientation="h",
                                   yanchor="bottom",
                                   y=1.01,
                                   xanchor="left",
                                   x=0,
                                   itemsizing='constant'),
                       coloraxis=dict(colorscale='viridis_r', colorbar_ticks='outside'))
    
    return plot

axis_options = list(data_types.header)
size_options = [i for i in axis_options if i != 'd34S']

base_errorbar = dict(type='data', thickness=1, color='DarkGrey')


def update_biv_traces(x, y, colour, size, diagonal, plot_errors, plot):
    
    with plot.batch_update():

        for category in categories:
            cat_df = log_df[log_df.category == category]

            if plot_errors:
                x_errorbars = {**base_errorbar, 'array': cat_df[x+'_err']}
                y_errorbars = {**base_errorbar, 'array': cat_df[y+'_err']}
            else:
                x_errorbars = None
                y_errorbars = None

            graph_texts = []
            for d in cat_df.iterrows():
                graph_texts.append(d[1]['name'] + '<br>' +
                                   data_types.loc[data_types.header==size, 'label'].iloc[0] + ': {:.2f}<br>'.format(d[1][size]) +
                                   data_types.loc[data_types.header==colour, 'label'].iloc[0] + ': {:.2f}'.format(d[1][colour]))

            cat_trace = sel_trace(plot, category)
            cat_trace.x=cat_df[x]
            cat_trace.y=cat_df[y]
            cat_trace.error_x=x_errorbars
            cat_trace.error_y=y_errorbars
            cat_trace.marker.size=cat_df[size]
            cat_trace.marker.sizemin=4
            cat_trace.marker.sizeref=3*np.mean(log_df[size]/(6.**2))
            cat_trace.marker.color=cat_df[colour]
            cat_trace.text=graph_texts


        if diagonal:
            line_bottom_left = min(log_df[x].min(), log_df[y].min())
            line_top_right = max(log_df[x].max(), log_df[y].max())
            plot.update_shapes(selector=dict(name='Diagonal'),
                               visible=True, x0 = line_bottom_left, y0 = line_bottom_left,
                               x1 = line_top_right, y1 = line_top_right)
        else:
            plot.update_shapes(selector=dict(name='Diagonal'), visible=False)
            
    
        # Only concentration axes should go to zero
        plot.layout.xaxis.rangemode = axis_mode(x)
        plot.layout.yaxis.rangemode = axis_mode(y)

        plot.layout.xaxis.title = data_types.loc[data_types.header==x, 'label'].iloc[0]
        plot.layout.yaxis.title = data_types.loc[data_types.header==y, 'label'].iloc[0]
        plot.layout.coloraxis.colorbar.title = data_types.loc[data_types.header==colour, 'label'].iloc[0]

In [6]:
# Customisable bivariate plot
biv_plot = init_biv_plot()

wdg.interact(update_biv_traces,
             x=wdg.Dropdown(options=axis_options, value='Fe_py'),
             y=wdg.Dropdown(options=axis_options, value='d34S'),
             colour=wdg.Dropdown(options=axis_options, value='height'),
             size=wdg.Dropdown(options=size_options, value='Constant'),
             diagonal=False,
             plot_errors=False,
             plot=wdg.fixed(biv_plot));

biv_plot

interactive(children=(Dropdown(description='x', index=3, options=('HR_to_tot', 'py_to_HR', 'py_to_tot', 'Fe_py…

FigureWidget({
    'data': [{'error_x': {},
              'error_y': {},
              'marker': {'color': arr…

In [7]:
# Fe speciation plot
spec_plot=init_biv_plot()
spec_plot.update_layout(xaxis_range=[0,1], yaxis_range=[0,1])
spec_plot.add_hline(0.8, line_width=1, line_dash='dash')
spec_plot.add_vline(0.38, line_width=1, line_dash='dash')

wdg.interact(update_biv_traces,
             x=wdg.fixed('HR_to_tot'),
             y=wdg.fixed('py_to_HR'),
             colour=wdg.Dropdown(options=axis_options, value='d34S'),
             size=wdg.Dropdown(options=size_options, value='Constant'),
             diagonal=wdg.fixed(False),
             plot_errors=False,
             plot=wdg.fixed(spec_plot));

spec_plot

interactive(children=(Dropdown(description='colour', index=10, options=('HR_to_tot', 'py_to_HR', 'py_to_tot', …

FigureWidget({
    'data': [{'error_x': {},
              'error_y': {},
              'marker': {'color': arr…

In [9]:
spec_plot.write_image('./images/spec_plot_4.pdf')

In [10]:
# Test model
HR_tot_0 = np.linspace(0.01,1,100)
py_HR_0 = np.zeros(50)

model_test=go.FigureWidget()
model_test.add_trace(go.Scatter(name='Constant q<sub>H2S</sub>', x=HR_tot_0))
#model_test.add_trace(go.Scatter(name='Decaying q<sub>H2S</sub>'))
model_test.update_layout(xaxis_range=[0,1], yaxis_range=[0,1],
                         width=700, height=700)
model_test.add_hline(0.8, line_width=1, line_dash='dash')
model_test.add_vline(0.38, line_width=1, line_dash='dash')
model_test.update_layout(xaxis_title='[Fe<sub>HR</sub>]/[Fe<sub>tot</sub>]',
                         yaxis_title='[Fe<sub>py</sub>]/[Fe<sub>HR</sub>]')


@wdg.interact(Fe_tot=wdg.FloatSlider(value=5, min=0, max=15, description='[Fe<sub>tot</sub>] / wt%', style=slider_style, layout=wdg.Layout(width='75%')),
              py_0 = wdg.FloatSlider(value=0, min=0, max=5, step=0.1, description='[Fe<sub>py</sub>]<sub>0</sub> / wt%', style=slider_style, layout=wdg.Layout(width='75%')),
              n_H2S=wdg.FloatSlider(value=0, min=0, max=1.5, step=0.01, description='H<sub>2</sub>S added / mol', style=slider_style, layout=wdg.Layout(width='75%')),
              plot=wdg.fixed(model_test))
def update_Fe_model(Fe_tot, py_0, n_H2S, plot):
    
    with plot.batch_update():
        
        d_py = M_to_wt(0.5*n_H2S)
        HR_0 = HR_tot_0 * Fe_tot

        py_HR_t = (py_0 + d_py)/HR_0

        model_line = sel_trace(plot, 'Constant q<sub>H2S</sub>')
        model_line.y = py_HR_t
    
    
model_test

interactive(children=(FloatSlider(value=5.0, description='[Fe<sub>tot</sub>] / wt%', layout=Layout(width='75%'…

FigureWidget({
    'data': [{'name': 'Constant q<sub>H2S</sub>',
              'type': 'scatter',
            …

In [11]:
# Modelling pyritisation of real data
psn_model=go.FigureWidget()

# for HR_to_tot in list(log_df.HR_to_tot):
#     psn_model.add_vline(HR_to_tot, line_width=0.5, line_color='Lavender')

for category in categories: 
    
    cat_df = log_df[log_df.category == category]

    graph_texts = []
    for d in cat_df.iterrows():
        graph_texts.append(d[1]['name'])
    
    # Current locations of samples in Fe speciation space
    
    # psn_model.add_trace(go.Scatter(x=cat_df['HR_to_tot'],
    #                                mode='markers',
    #                                marker_symbol=cat_symbols[category],
    #                                marker_color='LightPink',
    #                                name=category+' (pre-diagenesis)',
    #                                text=graph_texts,
    #                                legendgroup=category))
    
    # psn_model.add_trace(go.Scatter(x=cat_df['HR_to_tot'],
    #                                mode='markers',
    #                                marker_symbol=cat_symbols[category],
    #                                marker_color='LawnGreen',
    #                                name=category+' (half way)',
    #                                marker_opacity=0.8,
    #                                text=graph_texts,
    #                                legendgroup=category))
    
    psn_model.add_trace(go.Scatter(x=cat_df['HR_to_tot'],
                                   mode='markers',
                                   marker_symbol=cat_symbols[category],
                                   marker_color='Green',
                                   name=category+' (modelled)',
                                   text=graph_texts,
                                   legendgroup=category))

    # psn_model.add_trace(go.Scatter(x=cat_df['HR_to_tot'], y=cat_df['py_to_HR'],
    #                                mode='markers',
    #                                marker_symbol=cat_symbols[category],
    #                                marker_color='Blue',
    #                                marker_opacity=0.3,
    #                                name=category+' (post-diagenesis)',
    #                                text=graph_texts,
    #                                legendgroup=category))    
    

# for model line
HR_tot_0 = np.linspace(0.01,1,50) # Increase last number for higher resolution
py_HR_0 = np.zeros(len(HR_tot_0))

# psn_model.add_trace(go.Scatter(name='model (const. total Fe)', x=HR_tot_0,
#                                line_color='LightGreen'))

psn_model.update_layout(xaxis_range=[0,1], yaxis_range=[0,1],
                        width=900, height=650)
psn_model.add_hline(0.8, line_width=1, line_dash='dash')
psn_model.add_vline(0.38, line_width=1, line_dash='dash')
psn_model.update_layout(xaxis_title='[Fe<sub>HR</sub>]/[Fe<sub>tot</sub>]',
                        yaxis_title='[Fe<sub>py</sub>]/[Fe<sub>HR</sub>]',
                        legend=dict(#itemsizing='constant',
                                    font_size=11))



@wdg.interact(Fe_tot=wdg.FloatSlider(value=5.11, min=0, max=15, description='[Fe<sub>tot</sub>] / wt%', style=slider_style, layout=wdg.Layout(width='75%')),
              py_0 = wdg.FloatSlider(value=0, min=0, max=5, step=0.1, description='[Fe<sub>py</sub>]<sub>0</sub> / wt%', style=slider_style, layout=wdg.Layout(width='75%')),
              n_H2S=wdg.FloatSlider(value=0, min=0, max=1.5, step=0.01, description='H<sub>2</sub>S added / mol', style=slider_style, layout=wdg.Layout(width='75%')),
              alter_py_0=wdg.Checkbox(value=False, description='Correct model by changing [Fe<sub>py</sub>]<sub>0</sub>', style=slider_style),
              plot=wdg.fixed(psn_model))
def update_Fe_data(Fe_tot, py_0, n_H2S, alter_py_0, plot):
    
    with plot.batch_update():
        
        # update model line
        d_py = M_to_wt(0.5*n_H2S)
        HR_0 = HR_tot_0 * Fe_tot

        py_HR_t = (py_0 + d_py)/HR_0

        # Update y coordinates of line
        # sel_trace(plot, 'model (const. total Fe)').y = py_HR_t


        # update data points
        for category in categories:
            cat_df = log_df[log_df.category == category]

            py_HR_points = list((py_0 + d_py)/cat_df['Fe_HR'])

            for i in range(len(py_HR_points)):
                if py_HR_points[i] > 1:
                    py_HR_points[i] = 1

            # Update y coordinates of modelled points
            sel_trace(plot, category+' (modelled)').y=py_HR_points

            # sel_trace(plot, category+' (half way)').y = [0.5* i for i in py_HR_points]

            if alter_py_0:

                py_0s = cat_df['py_to_HR'] - py_HR_points

                # sel_trace(plot, category+' (pre-diagenesis)').y=py_0s
            else:
                # sel_trace(plot, category+' (pre-diagenesis)').y=py_0/cat_df['Fe_HR']
                pass
    
#     if alter_py_0:
#         plot.update_layout(yaxis_range=[-0.5,1])
#     else:
#         plot.update_layout(yaxis_range=[0,1])
    
    
psn_model

interactive(children=(FloatSlider(value=5.11, description='[Fe<sub>tot</sub>] / wt%', layout=Layout(width='75%…

FigureWidget({
    'data': [{'legendgroup': 'Mudstone',
              'marker': {'color': 'Green', 'symbol': '…

In [15]:
# Increase marker size for export
for trace in psn_model.data:
    trace.marker.size = 12

In [17]:
psn_model.write_image('./images/Fe_model_0-6M.pdf')

In [48]:
# Plot contours 
HR_tot_0 = np.linspace(0.01,1,100)

# create base graph
contour_model=go.Figure()
contour_model.add_hline(0.8, line_width=1, line_dash='dash')
contour_model.add_vline(0.38, line_width=1, line_dash='dash')

# Add original data
for category in categories: 
    
    cat_df = log_df[log_df.category == category]

    graph_texts = []
    for d in cat_df.iterrows():
        graph_texts.append(d[1]['name'])
    
 
    contour_model.add_trace(go.Scatter(x=cat_df['HR_to_tot'], y=cat_df['py_to_HR'],
                                       mode='markers',
                                       marker_symbol=cat_symbols[category],
                                       marker_color='Green',
                                       marker_opacity=0.4,
                                       marker_size=10,
                                       name=category+' (post-diagenesis)',
                                       text=graph_texts,
                                       legendgroup=category))    

# Find color scale
py_tots = np.arange(0, 1.01, 0.1)
color_list = pcol.sample_colorscale('sunset_r', py_tots)

# Loop over different Fepy/Fetot ratios
for n, py_tot in enumerate(py_tots):
    py_HR = [min([py_tot/ht, 1]) for ht in  HR_tot_0]
    contour_model.add_trace(go.Scatter(x=HR_tot_0, y=py_HR, name=f'py/tot = {py_tot:.1f}', text=f'{py_tot:.1f}', marker_color = color_list[n], opacity=0.7, mode='lines'))
    
contour_model.update_layout(xaxis_range=[0,1], yaxis_range=[0,1],width=650, height=650,
                            xaxis_title='[Fe<sub>HR</sub>]/[Fe<sub>tot</sub>]',
                            yaxis_title='[Fe<sub>py</sub>]/[Fe<sub>HR</sub>]',
                            showlegend=False)

contour_model.show()

In [49]:
contour_model.write_image('./images/Fe_contours.pdf')

In [13]:
# Running the model in reverse to look at effect of weathering Fepy → FeHR

# Plot contours 
HR_tot_0 = np.linspace(0.01,1,100)

# create base graph
contour_model=go.Figure()
contour_model.add_hline(0.8, line_width=1, line_dash='dash')
contour_model.add_vline(0.38, line_width=1, line_dash='dash')

# Add original data
for category in categories: 
    
    cat_df = log_df[log_df.category == category]

    graph_texts = []
    for d in cat_df.iterrows():
        graph_texts.append(d[1]['name'])
    
 
    contour_model.add_trace(go.Scatter(x=cat_df['HR_to_tot'], y=cat_df['py_to_HR'],
                                       mode='markers',
                                       marker_symbol=cat_symbols[category],
                                       marker_color='Green',
                                       marker_opacity=0.4,
                                       marker_size=10,
                                       name=category+' (post-diagenesis)',
                                       text=graph_texts,
                                       legendgroup=category))    

# Find color scale
py_tots = np.arange(0, 1.01, 0.1)
color_list = pcol.sample_colorscale('sunset_r', py_tots)

# Loop over different Fepy/Fetot ratios
for n, py_tot in enumerate(py_tots):
    py_HR = [max([1- py_tot/ht, 0]) for ht in  HR_tot_0]
    contour_model.add_trace(go.Scatter(x=HR_tot_0, y=py_HR, name=f'py/tot = {py_tot:.1f}', text=f'{py_tot:.1f}', marker_color = color_list[n], opacity=0.7, mode='lines'))
    
contour_model.update_layout(xaxis_range=[0,1], yaxis_range=[0,1],width=650, height=650,
                            xaxis_title='[Fe<sub>HR</sub>]/[Fe<sub>tot</sub>]',
                            yaxis_title='[Fe<sub>py</sub>]/[Fe<sub>HR</sub>]',
                            showlegend=False)

contour_model.show()

In [14]:
contour_model.write_image('./images/weathering_contours.pdf')